In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../src")

In [2]:
import mesh
from tqdm.autonotebook import tqdm
from CwnGraph import CwnBase
import json, pickle

<ipython-input-2-1f5aa210c2a3>:2: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [3]:
import requests

In [8]:
with (mesh.get_data_dir()/"babelnet_key.txt").open("r") as fin:
    bn_key = fin.read().strip()
bn_url = "https://babelnet.io/v5/"

In [6]:
bn_key

'22928647-9105-4264-88be-387fc7831fcc'

In [13]:
bnapi = mesh.BabelNetAPI()

In [14]:
bnapi.get_version()

'V4_0'

In [25]:
cwn = CwnBase()

In [32]:
lemmas = []
for nid, ndata in cwn.V.items():
    if ndata["node_type"] == "lemma":        
        lemmas.append(ndata["lemma"])

In [42]:
from itertools import chain
def find_cwn_senses(lemma):
    try:
        sense_iter = (x.senses for x in cwn.find_lemma(f"^{lemma}$"))
        sense_iter = chain.from_iterable(sense_iter)
        return list(sense_iter)
    except Exception as ex:
        print(lemma)
        print(ex)
        return []

In [58]:
target_lemmas = []
for lemma_x in tqdm(set(lemmas)):
    if not lemma_x:
        continue    
    try:        
        sense_list = find_cwn_senses(lemma_x)        
        if len(sense_list) > 2:
            target_lemmas.append(lemma_x)
    except Exception as ex:
        print(ex)        

In [63]:
target_lemma_path = mesh.get_data_dir()/"bn_target_lemmas.txt"
with target_lemma_path.open("w", encoding="UTF-8") as fout:
    for lemma in target_lemmas:
        fout.write(lemma + "\n")

In [ ]:
target_lemma_path = mesh.get_data_dir()/"bn_target_lemmas.txt"
with target_lemma_path.open("r", encoding="UTF-8") as fin:
    taget_lemmas = fin.readlines()

## Continue to retrieve data from BabelNet

In [64]:
target_lemma_path = mesh.get_data_dir()/"bn_target_lemmas.txt"
with target_lemma_path.open("r", encoding="UTF-8") as fin:
    taget_lemmas = fin.readlines()
    
bn_sense_path = mesh.get_data_dir()/"bn_sense_data.pkl"
if bn_sense_path.exists():
    with bn_sense_path.open("r") as fin:
        sense_data = pickle.load(fin)
else:
    sense_data = {}

In [76]:
quota = 50
for lemma_x in tqdm(target_lemmas):
    if quota == 0:
        break
    if lemma_x in sense_data:
        continue
    data = bnapi.get_senses(lemma_x)
    quota -= 1
    sense_data[lemma_x] = data    

In [77]:
len(sense_data)

950

In [78]:
with bn_sense_path.open("wb") as fout:
    pickle.dump(sense_data, fout)

In [83]:
s1 = list(sense_data.items())[1]

In [90]:
wn_map = {}
for lemma, senses in sense_data.items():    
    if not senses:
        continue
    wn_senses = filter(lambda x: x["type"] == "WordNetSense", senses)
    slist = []
    for s in wn_senses:
        p = s["properties"]
        slist.append((p.get("wordNetOffset"), 
                     p.get("fullLemma"), 
                     p.get("senseKey"), p.get("pos"), p.get("synsetID").get("id"), p.get("bKeySense")))
    wn_map[lemma] = slist

In [94]:
print(f"target_lemmas (>3 in CWN): {len(target_lemmas)}\n"
      f"sense_data (requested from BN so far): {len(sense_data)}\n"
      f"with WN senses: {len(wn_map)}")

target_lemmas (>3 in CWN): 3396
sense_data (requested from BN so far): 950
with WN senses: 711


## A word sense sample

In [96]:
from nltk.corpus import wordnet as wn

In [97]:
wn_map["犯"]

[('02582615v',
  'perpetrate',
  'perpetrate%2:41:00::',
  'VERB',
  'bn:00085426v',
  False),
 ('02582615v', 'commit', 'commit%2:41:00::', 'VERB', 'bn:00085426v', False),
 ('02582615v', 'pull', 'pull%2:41:00::', 'VERB', 'bn:00085426v', False),
 ('02582921v', 'make', 'make%2:41:13::', 'VERB', 'bn:00090561v', False)]

In [105]:
wn.synset_from_pos_and_offset("v", 2582615).definition()

'perform an act, usually with a negative connotation'

In [106]:
wn.synset_from_pos_and_offset("v", 2582921).definition()

'carry out or commit'

In [107]:
find_cwn_senses("犯")

[<CwnSense[06700001](犯): 越過本身應有的範圍，而侵占到後述對象的權益。>,
 <CwnSense[06700002](犯): 不遵守約定或上級規定的規則。>,
 <CwnSense[06700003](犯): 違反競賽的規則，通常由裁判所判定，並帶有一定的罰則。>,
 <CwnSense[06700004](犯): 做出不應該或不正確的事情。>,
 <CwnSense[06700005](犯): 做出違反法律、應受處罰的事情。>,
 <CwnSense[06700006](犯): 做出違反法律的事情，而應受處罰的人。>,
 <CwnSense[06700007](犯): 做出特定被禁止的行為的事情。>,
 <CwnSense[06700008](犯): 疾病病徵再次發生。>,
 <CwnSense[06700009](犯): 侵犯到後述對象的權益而遭到報復。>]